In [43]:
import pandas as pd
import sys
import os

sys.path.append("..")
from data_generation_scripts.utils import check_rate_limit
from data_generation_scripts.generate_org_users_interactions import get_org_users_activities
from data_generation_scripts.generate_org_metadata import get_counts
from data_generation_scripts.generate_org_repos_interactions import get_org_repo_activities

In [2]:
rates_df = check_rate_limit()

### Get Initial Orgs

In [30]:
initial_core_orgs = pd.read_csv("../data/derived_files/initial_core_orgs.csv")

In [3]:
core_orgs_path = "../data/derived_files/firstpass_core_orgs.csv"
core_orgs = pd.read_csv(core_orgs_path)

In [33]:
combined_core_orgs = pd.concat([initial_core_orgs, core_orgs], ignore_index=True)

In [6]:
user_df = pd.read_csv("../data/large_files/entity_files/users_dataset.csv")

In [38]:
orgs_df = pd.read_csv("../data/entity_files/orgs_dataset.csv")

In [48]:
user_df.columns

Index(['login', 'id', 'node_id', 'avatar_url', 'gravatar_id', 'url',
       'html_url', 'followers_url', 'following_url', 'gists_url',
       'starred_url', 'subscriptions_url', 'organizations_url', 'repos_url',
       'events_url', 'received_events_url', 'type', 'site_admin', 'name',
       'company', 'blog', 'location', 'email', 'hireable', 'bio',
       'twitter_username', 'public_repos', 'public_gists', 'followers',
       'following', 'created_at', 'updated_at', 'user_query_time'],
      dtype='object')

In [49]:
subset_user = user_df[user_df['type'] == "Organization"]

In [50]:
all_orgs = pd.concat([orgs_df, subset_user])

In [51]:
len(all_orgs), len(combined_core_orgs)

(3354, 460)

In [52]:
org_headers = pd.read_csv("../data/metadata_files/org_headers.csv")

In [46]:
'bio' in subset_user.columns

True

In [55]:
orgs_df[org_headers.columns].columns.tolist()

['login',
 'id',
 'node_id',
 'url',
 'repos_url',
 'events_url',
 'hooks_url',
 'issues_url',
 'members_url',
 'public_members_url',
 'avatar_url',
 'description',
 'name',
 'company',
 'blog',
 'location',
 'email',
 'twitter_username',
 'is_verified',
 'has_organization_projects',
 'has_repository_projects',
 'public_repos',
 'public_gists',
 'followers',
 'following',
 'html_url',
 'created_at',
 'updated_at',
 'type']

In [53]:
set(subset_user.columns.tolist()) & set(orgs_df.columns.tolist())

{'avatar_url',
 'blog',
 'company',
 'created_at',
 'email',
 'events_url',
 'followers',
 'following',
 'html_url',
 'id',
 'location',
 'login',
 'name',
 'node_id',
 'public_gists',
 'public_repos',
 'repos_url',
 'twitter_username',
 'type',
 'updated_at',
 'url'}

In [36]:
all_orgs

,login,id,node_id,url,repos_url,events_url,hooks_url,issues_url,members_url,public_members_url,...,following_url,gists_url,starred_url,subscriptions_url,organizations_url,received_events_url,site_admin,hireable,bio,user_query_time
0,dhc-barnard,69482009.0,MDEyOk9yZ2FuaXphdGlvbjY5NDgyMDA5,https://api.github.com/orgs/dhc-barnard,https://api.github.com/orgs/dhc-barnard/repos,https://api.github.com/orgs/dhc-barnard/events,https://api.github.com/orgs/dhc-barnard/hooks,https://api.github.com/orgs/dhc-barnard/issues,https://api.github.com/orgs/dhc-barnard/member...,https://api.github.com/orgs/dhc-barnard/public...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,irLh,28738013.0,MDEyOk9yZ2FuaXphdGlvbjI4NzM4MDEz,https://api.github.com/orgs/irLh,https://api.github.com/orgs/irLh/repos,https://api.github.com/orgs/irLh/events,https://api.github.com/orgs/irLh/hooks,https://api.github.com/orgs/irLh/issues,https://api.github.com/orgs/irLh/members{/member},https://api.github.com/orgs/irLh/public_member...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ABC-DH,55312416.0,MDEyOk9yZ2FuaXphdGlvbjU1MzEyNDE2,https://api.github.com/orgs/ABC-DH,https://api.github.com/orgs/ABC-DH/repos,https://api.github.com/orgs/ABC-DH/events,https://api.github.com/orgs/ABC-DH/hooks,https://api.github.com/orgs/ABC-DH/issues,https://api.github.com/orgs/ABC-DH/members{/me...,https://api.github.com/orgs/ABC-DH/public_memb...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,dhhse,55768668.0,MDEyOk9yZ2FuaXphdGlvbjU1NzY4NjY4,https://api.github.com/orgs/dhhse,https://api.github.com/orgs/dhhse/repos,https://api.github.com/orgs/dhhse/events,https://api.github.com/orgs/dhhse/hooks,https://api.github.com/orgs/dhhse/issues,https://api.github.com/orgs/dhhse/members{/mem...,https://api.github.com/orgs/dhhse/public_membe...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,YaleDHLab,17452720.0,MDEyOk9yZ2FuaXphdGlvbjE3NDUyNzIw,https://api.github.com/orgs/YaleDHLab,https://api.github.com/orgs/YaleDHLab/repos,https://api.github.com/orgs/YaleDHLab/events,https://api.github.com/orgs/YaleDHLab/hooks,https://api.github.com/orgs/YaleDHLab/issues,https://api.github.com/orgs/YaleDHLab/members{...,https://api.github.com/orgs/YaleDHLab/public_m...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126872,matrix-msu,6820823.0,MDEyOk9yZ2FuaXphdGlvbjY4MjA4MjM=,https://api.github.com/users/matrix-msu,https://api.github.com/users/matrix-msu/repos,https://api.github.com/users/matrix-msu/events...,NaN,NaN,NaN,NaN,...,https://api.github.com/users/matrix-msu/follow...,https://api.github.com/users/matrix-msu/gists{...,https://api.github.com/users/matrix-msu/starre...,https://api.github.com/users/matrix-msu/subscr...,https://api.github.com/users/matrix-msu/orgs,https://api.github.com/users/matrix-msu/receiv...,False,NaN,NaN,2023-06-15
126932,fork-maintainers,70783733.0,MDEyOk9yZ2FuaXphdGlvbjcwNzgzNzMz,https://api.github.com/users/fork-maintainers,https://api.github.com/users/fork-maintainers/...,https://api.github.com/users/fork-maintainers/...,NaN,NaN,NaN,NaN,...,https://api.github.com/users/fork-maintainers/...,https://api.github.com/users/fork-maintainers/...,https://api.github.com/users/fork-maintainers/...,https://api.github.com/users/fork-maintainers/...,https://api.github.com/users/fork-maintainers/...,https://api.github.com/users/fork-maintainers/...,False,NaN,NaN,2023-06-15
126968,TNO-MPC,80950308.0,MDEyOk9yZ2FuaXphdGlvbjgwOTUwMzA4,https://api.github.com/users/TNO-MPC,https://api.github.com/users/TNO-MPC/repos,https://api.github.com/users/TNO-MPC/events{/p...,NaN,NaN,NaN,NaN,...,https://api.github.com/users/TNO-MPC/following...,https://api.github.com/users/TNO-MPC/gists{/gi...,https://api.github.com/users/TNO-MPC/starred{/...,https://api.github.com/users/TNO-MPC/subscript...,https://api.github.com/users/TNO-MPC/orgs,https://api.github.com/users/TNO-MPC/received_...,False,NaN,TNO - MPC Lab,2023-06-15
127110,PersDigUMD,13054030.0,

### Get Missing Counts

In [4]:
if os.path.exists("../data/metadata_files/user_url_cols.csv"):
    cols_df = pd.read_csv("../data/metadata_files/user_url_cols.csv")
else:
    cols_dict ={'followers': 'followers_url', 'following': 'following_url', 'public_repos': 'repos_url', 'public_gists': 'gists_url', 'star_count': 'starred_url', 'subscription_count': 'subscriptions_url', 'organization_count': 'organizations_url'}
    cols_df = pd.DataFrame(cols_dict.items(), columns=['col_name', 'col_url'])
    cols = cols_df.col_name.tolist()
    reverse_cols = cols[::-1]
    cols_df.to_csv("../data/metadata_files/user_url_cols.csv", index=False)

In [5]:
core_orgs["members_url"] = core_orgs["url"].apply(lambda x: x + "/public_members")
core_orgs.members_url = core_orgs.members_url.str.replace('users', 'orgs')
for index, row in cols_df.iterrows():
    if (row['col_name'] not in core_orgs.columns) or (core_orgs[core_orgs[row.col_name].isna()].shape[0] > 0):
        if 'url' in row.col_url:
            print(f'Getting {row.col_name} for core orgs')
            core_orgs = get_counts(core_orgs, row.col_url, row.col_name, overwrite_existing_temp_files=False)
            core_orgs.to_csv(core_orgs_path, index=False)
        else:
            print(f'Issues with {row.col_name} for core orgs')
                

### Get Organization Members

In [19]:
org_members_output_path = "../data/join_files/org_members_join_dataset.csv"
users_output_path = "../data/entity_files/users_dataset.csv"
get_url_field = "members_url"
load_existing_files = True
overwrite_existing_temp_files = False
join_unique_field = "org_login"
filter_fields = ["org_login", "login"]
retry_error = False

org_members_df, user_df = get_org_users_activities(core_orgs,org_members_output_path, users_output_path, get_url_field, load_existing_files, overwrite_existing_temp_files, join_unique_field, filter_fields, retry_error)

### Get Organization Followers

In [ ]:
org_followers_output_path = "../data/join_files/org_followers_join_dataset.csv"
users_output_path = "../data/entity_files/users_dataset.csv"
get_url_field = "followers_url"
load_existing_files = True
overwrite_existing_temp_files = False
join_unique_field = "org_login"
filter_fields = ["org_login", "login"]
retry_error = False
org_followers_df, user_df = get_org_users_activities(core_orgs,org_followers_output_path, users_output_path, get_url_field, load_existing_files, overwrite_existing_temp_files, join_unique_field, filter_fields, retry_error)

### Get Organization Repos

In [32]:
org_repos_output_path = "../data/join_files/org_repos_join_dataset.csv"
repo_output_path = "../data/large_files/entity_files/repo_dataset.csv"
get_url_field = "repos_url"
load_existing_files = True
overwrite_existing_temp_files = False
join_unique_field = "org_login"
filter_fields = ["org_login", "full_name"]
retry_error = True
org_repos_df, user_df = get_org_repo_activities(core_orgs,org_repos_output_path, repo_output_path, get_url_field, load_existing_files, overwrite_existing_temp_files, join_unique_field, filter_fields, retry_error)